# 03) 모델 학습(체크포인트) & 저장 / 재사용 추론
- AutoEncoder(MSE 재구성 오차) 기반 비지도 학습.
- 학습 중 매 에폭 체크포인트 저장, 최저 val-loss 기준 best 모델 저장.
- 같은 모델을 재사용해 다른 주차 데이터에 대해 추론(스코어 산출) 가능.

In [ ]:

BASE_DIR = r"C:\Users\EL040\Desktop\MS_3rd-Project\basemodel"
TRAIN_SPLIT = "train"   # 학습에 쓸 split
TEST_SPLIT  = "test"    # 추론(평가)에 쓸 split (동일 주차 반복 시 바꾸면 됨)
EPOCHS = 20
BATCH = 1024
LR = 1e-3
VAL_RATIO = 0.1


In [ ]:

import os
import numpy as np
import pandas as pd
from tqdm import tqdm

from common import get_paths, train_autoencoder, score_anomaly, load_scalers

paths = get_paths(BASE_DIR)
model_dir = paths["model_dir"]
ckpt_dir = paths["ckpt_dir"]

# Load TRAIN features
train_feat_path = os.path.join(paths["data_train_feat"], "features.parquet")
train_df = pd.read_parquet(train_feat_path)
X_train = train_df.values.astype("float32")
print("Train features:", X_train.shape)

# Train model with checkpoints
best_model_path, last_ckpt_path = train_autoencoder(
    X=X_train,
    model_dir=model_dir,
    ckpt_dir=ckpt_dir,
    epochs=EPOCHS,
    batch_size=BATCH,
    lr=LR,
    val_ratio=VAL_RATIO,
    tqdm_ncols=30,
)
print("Best model saved at:", best_model_path)
print("Last checkpoint:", last_ckpt_path)


In [ ]:

# ----- 재사용: 다른 데이터셋(예: TEST_SPLIT)의 스코어 산출 -----
test_feat_dir = paths["data_test_feat"]
test_feat_path = os.path.join(test_feat_dir, "features.parquet")
if os.path.exists(test_feat_path):
    test_df = pd.read_parquet(test_feat_path)
    X_test = test_df.values.astype("float32")
    print("Test features:", X_test.shape)

    scores = score_anomaly(best_model_path, X_test, tqdm_ncols=30)
    out_csv = os.path.join(test_feat_dir, "anomaly_scores.csv")
    pd.DataFrame({"anomaly_score": scores}).to_csv(out_csv, index=False)
    print("Saved test anomaly scores to:", out_csv)
else:
    print("No test features found. You can generate them with notebook 02 for TEST split.")
